In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
from pathlib import Path

import torch

baseline_path = Path("/bigdata/cifar10/logs/garipov/baselines/1647358615/gar_18_dNone_128_sgd_l0.1_g0.1_w0.0_sTrue")
baseline_model = torch.load(baseline_path / "cnn_best.pth")
with open(baseline_path/'results.json') as json_file:
    baseline_result = json.load(json_file)
baseline_result

/home/jetzeschuurman/gitProjects/phd/tltorch/tltorch/factorized_tensors/core.py:145: UserWarning: Creating a subclass of FactorizedTensor TensorizedTensor with no name.
  warnings.warn(f'Creating a subclass of FactorizedTensor {cls.__name__} with no name.')


{'best_epoch': 38,
 'best_train_acc': 0.9749333333333333,
 'best_valid_acc': 0.9078,
 'best_valid_loss': 0.003568341651931405,
 'test_acc': 0.891,
 'test_loss': 0.004007616302371025,
 'n_param': 557642,
 'model_name': 'gar_18_dNone_128_sgd_l0.1_g0.1_w0.0_sTrue'}

In [3]:
baseline_model

GaripovNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4_bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5_bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv6): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=128, out_features=1

In [4]:
import copy
from tddl.utils.model_stats import count_parameters
from tddl.factorizations import number_layers, listify_numbered_layers, get_weights
from tddl.dbs import find_error_given_c

baseline_count = count_parameters(baseline_model)
model = copy.deepcopy(baseline_model)
gar_layers = [2,4,6,8,10]
rank = 0.5

numbered_layers = number_layers(model)
listed_layers = listify_numbered_layers(numbered_layers, layer_nrs=gar_layers)
rank, c, error = find_error_given_c(listed_layers, desired_c = rank, baseline_count=baseline_count)

0.5


/home/jetzeschuurman/gitProjects/phd/tddl/venv/lib/python3.8/site-packages/tensorly/backend/core.py:885: UserWarning: In partial_svd: converting to NumPy. Check SVD_FUNS for available alternatives if you want to avoid this.
  warnings.warn('In partial_svd: converting to NumPy.'


factorize_layer.cache_info() = CacheInfo(hits=0, misses=6, maxsize=4096, currsize=6)
count_parameters.cache_info() = CacheInfo(hits=0, misses=2, maxsize=128, currsize=2)
factorize_layer.cache_info() = CacheInfo(hits=0, misses=11, maxsize=4096, currsize=11)
count_parameters.cache_info() = CacheInfo(hits=0, misses=3, maxsize=128, currsize=3)
factorize_layer.cache_info() = CacheInfo(hits=0, misses=15, maxsize=4096, currsize=15)
count_parameters.cache_info() = CacheInfo(hits=0, misses=4, maxsize=128, currsize=4)
factorize_layer.cache_info() = CacheInfo(hits=0, misses=18, maxsize=4096, currsize=18)
count_parameters.cache_info() = CacheInfo(hits=0, misses=5, maxsize=128, currsize=5)
factorize_layer.cache_info() = CacheInfo(hits=0, misses=26, maxsize=4096, currsize=26)
count_parameters.cache_info() = CacheInfo(hits=0, misses=6, maxsize=128, currsize=6)
find_rank_given_error.cache_info() = CacheInfo(hits=0, misses=5, maxsize=128, currsize=5)
0.2735034305163528
0.22649656948364721
True
--------

In [ ]:
# do constant compression

# get the approximation error

# get the min and max of the approximation error

# set min and max

In [6]:
from tddl.factorizations import number_layers, listify_numbered_layers, get_weights

numbered_layers = number_layers(baseline_model)
gar_layers = [2,4,6,8,10]
layers = listify_numbered_layers(numbered_layers, layer_nrs=gar_layers)

In [8]:
import numpy as np
from tddl.factorizations import factorize_layer

DESIRED_COMPRESSION = 0.5

rank = DESIRED_COMPRESSION
errors = np.array([])
for layer in layers:
    with torch.no_grad():
        fact_layer, error = factorize_layer(layer[2], 'tucker', rank, return_error=True)
    errors = np.append(errors, float(error.cpu().numpy()))

/home/jetzeschuurman/gitProjects/phd/tddl/venv/lib/python3.8/site-packages/tensorly/backend/core.py:885: UserWarning: In partial_svd: converting to NumPy. Check SVD_FUNS for available alternatives if you want to avoid this.
  warnings.warn('In partial_svd: converting to NumPy.'


In [9]:
errors

array([0.31196895, 0.36436999, 0.46476951, 0.42094678, 0.1999761 ])

In [10]:
error = errors.mean()
max_error = errors.max()
min_error = errors.min()

In [13]:
error

0.35240626633167266

In [15]:
find_error_given_c(layers, desired_c = DESIRED_COMPRESSION, error=error, max_error=max_error, min_error=min_error, baseline_count=baseline_count)

0.35240626633167266


/home/jetzeschuurman/gitProjects/phd/tddl/venv/lib/python3.8/site-packages/tensorly/backend/core.py:885: UserWarning: In partial_svd: converting to NumPy. Check SVD_FUNS for available alternatives if you want to avoid this.
  warnings.warn('In partial_svd: converting to NumPy.'


factorize_layer.cache_info() = CacheInfo(hits=91, misses=66, maxsize=4096, currsize=66)
count_parameters.cache_info() = CacheInfo(hits=5, misses=32, maxsize=128, currsize=32)
factorize_layer.cache_info() = CacheInfo(hits=92, misses=70, maxsize=4096, currsize=70)
count_parameters.cache_info() = CacheInfo(hits=5, misses=33, maxsize=128, currsize=33)
factorize_layer.cache_info() = CacheInfo(hits=93, misses=73, maxsize=4096, currsize=73)
count_parameters.cache_info() = CacheInfo(hits=5, misses=34, maxsize=128, currsize=34)
factorize_layer.cache_info() = CacheInfo(hits=94, misses=75, maxsize=4096, currsize=75)
count_parameters.cache_info() = CacheInfo(hits=5, misses=35, maxsize=128, currsize=35)
factorize_layer.cache_info() = CacheInfo(hits=95, misses=80, maxsize=4096, currsize=80)
count_parameters.cache_info() = CacheInfo(hits=5, misses=36, maxsize=128, currsize=36)
find_rank_given_error.cache_info() = CacheInfo(hits=0, misses=5, maxsize=128, currsize=5)
0.46897113201659846
0.0310288679834

([0.4375, 0.5625, 0.75, 0.65625, 0.09375],
 0.5011727954494102,
 tensor(0.3323, device='cuda:0'))